In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import os
import random
import matplotlib

from matplotlib import rc
from matplotlib.patches import Ellipse
from scipy import stats

rc('text', usetex=False)
rc('font', family='Arial')

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import warnings
warnings.filterwarnings('ignore')

## Get list of targets

In [ ]:
targets = pd.read_csv('https://predictioncenter.org/casp14/targetlist.cgi?type=csv', sep = ';', error_bad_lines = False)
targets

In [ ]:
cancelled_targets = []
for index, row in targets.iterrows():
    if 'canceled' in row.Description or 'Canceled' in row.Description:
        cancelled_targets.append(row.Target)

# Load tables

In [ ]:
def load_casp_zscore_table(target_casp, cancelled_targets, mode = None, domains_only = False):
    
    if mode is not None:
        table = 'CASP{}/CASP{}_templates_table.csv'.format(target_casp, target_casp)
    else:
        table = 'CASP{}/CASP{}_Zscores_table.csv'.format(target_casp, target_casp)

    df = pd.read_csv(table, sep='\t').reset_index(drop=True)
    
    if domains_only:
        # remove the multidomain lines
        df = df[df.Target.str.contains("-D")]

    # remove the NaN models
    df = df[df['Model'].notna()]
    
    # add model ranking based on group
    if mode is None:            
        df['Model_GR_rank'] = [int(model.split('_')[-1].split('-')[0]) if model != 'Starting' else 0 for model in df.Model]
    else:
        # for the template tables, add as the group either that they are template, or the real structure
        df['GR#'] = ['TMP' if type(i) == str else 'REAL' for i in df.pdb]
                
    #df['Input_target'] = [target.replace('R', 'T') for target in df.Target]
    
    # exclude the cancelled targets    
    df = df[~df['Target'].isin(cancelled_targets)]
    df = df[df['Target'].notna()]

    
    return df

In [ ]:
target_casp = '14'
domains_only = True
do_null = True

if domains_only:
    figures_folder = 'CASP{}/highres_figs_EXCLUDING_MULTIDOMAINS'.format(target_casp)
else:
    figures_folder = 'CASP{}/highres_figs'.format(target_casp)

if not do_null:
    figures_folder = '{}_NO_NULL'.format(figures_folder)
    
if not os.path.isdir(figures_folder):
    os.mkdir(figures_folder)

df = load_casp_zscore_table(target_casp, cancelled_targets, domains_only = domains_only)
df_templates = load_casp_zscore_table(target_casp, cancelled_targets, domains_only = domains_only, mode = 'template')

# START PLOTTING

In [ ]:
# rename the columns of the templates table
new_names = []
for col in df_templates.columns:
    if '_GD' in col:
        col = col.replace('Template_', '')
    elif 'Template' in col and '_score' not in col:
        col = col.replace('Template', 'Model')
    new_names.append(col)
df_templates.columns = new_names

df_templates = df_templates.set_index('Model')
df_templates

## 1. Backbone and sidechain modelling analysis

### ***This analysis was carried out***:
 - only accounting with the 1st models submitted by each group and ignored refined models!
 - for groups that submitted models for at least 10 targets

In [ ]:
df_1st = df.loc[(df.Model_GR_rank == 1) & (df.Model.str.startswith('T'))]
#df_1st = df.loc[(df.Model_GR_rank == 1)]
df_1st = df_1st.groupby(['GR#']).filter(lambda s: s['GR#'].count()>=10)
df_1st

### 1.1. Backbone geometry quality 

#### 1.1.1. Target boxplots for backbone scores


In [ ]:
def targets_boxplot(df_1st, templates_df, target_parameter):
    
    sorted_nb = df_1st.groupby(['Target'])[target_parameter].median().sort_values(ascending=False)
    
    print('Median median {}:'.format(target_parameter), sorted_nb.median())
    
#     plt.figure(figsize=(15, 5))
#     g = sns.boxplot(x='Target', y=target_parameter, color = 'whitesmoke', data = df_1st, fliersize=2, dodge=False, order=sorted(list(sorted_nb.index)), linewidth=1)
#     g.axhline(0, linestyle=':', color='black')
    
#     s = sns.stripplot(x='Target', y=target_parameter, hue = 'Template_score', palette= 'Greens', linewidth=0.3, size = 5, data = df_templates.loc[df_templates['GR#'] == 'TMP'], order=sorted(list(sorted_nb.index)))
#     #s = sns.stripplot(x='Target', y=target_parameter, color='orange', linewidth=0.3, size = 5, data = df_1st.loc[df_1st['GR#'] == '427'], order=sorted(list(sorted_nb.index)))
#     s.get_legend().remove()
    
#     plt.xticks(rotation=90)
#     plt.title('General boxplot of Targets {}'.format(target_parameter))
    
#     plt.tight_layout()
#     plt.savefig("{}/targets_boxplot_{}.pdf".format(figures_folder, target_parameter))

#     plt.show()
#     plt.close()

In [ ]:
target_parameters = ['Model_DipDiff','Model_BBscore', 'GDT_HA']
for p in target_parameters:
    targets_boxplot(df_1st, df_templates, p)

In [ ]:
target_parameters = ['Model_DipDiff']
compare_parameters = ['GDT_HA', 'Model_BBscore']
for p in target_parameters:   
    for i in compare_parameters:
    
        plt.figure(figsize=(3.5,3))
        plt.scatter(df_1st[p], df_1st[i], s=1, c='silver', label='Models')
        #plt.scatter(df_1st[df_1st['GR#'] == '427'][p], df_1st[df_1st['GR#'] == '427'][i], s=1, c='orange', label='AlphaFold2')
        try:
            plt.scatter(df_templates[p], df_templates[i], s=1, c='teal', label = 'Templates')
        except:
            pass
        plt.xlabel('{}'.format(p))
        if 'GD' in i:
            ylab = 'Model_{}'.format(i)
        else:
            ylab = i
            
        plt.ylabel(i)
        plt.legend()
        
        if 'GD' in i:
            ymax = 100
        else:
            ymax = max(df_1st[i])
        plt.ylim(0, ymax)    
        plt.vlines(x=0, linestyle=':', color='k', ymin=0, ymax = 100)

        plt.title('PCC for {} models: {}'.format(len(df_1st), round(df_1st.corr()[p][i], 3)))

        plt.tight_layout()
        plt.savefig("{}/scatter_{}_vs_{}.pdf".format(figures_folder, p, ylab))
        plt.show()


In [ ]:
print('Percentage of models with DipDiff > 0: {}%'.format(len(df_1st.loc[df_1st.Model_DipDiff > 0])*100/len(df_1st)))
print('Percentage of models with DipDiff > 0.1: {}%'.format(len(df_1st.loc[df_1st.Model_DipDiff > 0.1])*100/len(df_1st)))
print('Percentage of models with DipDiff < 0: {}%'.format(len(df_1st.loc[df_1st.Model_DipDiff < 0])*100/len(df_1st)))
print('Percentage of models with DipDiff = 0: {}%'.format(len(df_1st.loc[df_1st.Model_DipDiff == 0])*100/len(df_1st)))
print()
print('Percentage of templates with DipDiff > 0: {}%'.format(round(len(df_templates.loc[df_templates.Model_DipDiff > 0])*100/len(df_templates))))

In [ ]:
df_1st_posDipDiff = df_1st.loc[df_1st.Model_DipDiff > 0.1]
plt.hist(df_1st_posDipDiff.GDT_HA, bins=range(0, 101, 2))
plt.xlabel('GDT_HA')
plt.ylabel('Frequency')
plt.title('GDT_HA distribution for models with DipDiff > 0.1')
plt.show()

### 1.2. Sidechain geometry quality 

#### 1.2.1. How is sidechain modelling affected by backbone geometric quality?


In [ ]:
target_parameters = ['Model_DipDiff', 'Model_BBscore']
for p in target_parameters:
    
    plt.figure(figsize=(3.5,3))
    plt.scatter(abs(df_1st[p]), df_1st.Model_SCscore, s=1, c='silver', label='Models')
    #plt.scatter(abs(df_1st[df_1st['GR#'] == '427'][p]), df_1st[df_1st['GR#'] == '427'].Model_SCscore, s=1, c='orange', label='AlphaFold2')
    plt.scatter(abs(df_templates[p]), df_templates.Model_SCscore, s=1, c='teal', label = 'Templates')
    if 'Dipdiff' in p:
        plt.xlabel('Absolute {}'.format(p))
    else:
        plt.xlabel(p)
    plt.ylabel('Model_SCscore')
    plt.legend()

    plt.title('PCC for {} models: {}'.format(len(df_1st), round(df_1st.corr()[p]['Model_SCscore'], 3)))
    
    plt.tight_layout()
    plt.savefig("{}/scatter_{}_vs_{}.pdf".format(figures_folder, p, 'Model_SCscore'))
    plt.show()


#### 1.2.2. Target boxplots for sidechain scores


In [ ]:
target_parameters = ['Model_SCscore', 'GDC_SC']
for p in target_parameters:
    targets_boxplot(df_1st, df_templates, p)

## 2. Overall group rankings based on different metrics

In [ ]:
def groups_boxplot(df_1st, target_parameter, target_color, topn = 10, palette = 'Blues_r'):
    
    sorted_nb = df_1st.groupby(['GR_name'])[target_parameter].median().sort_values(ascending=False)
    
    df_1st[target_color].fillna('?', inplace=True)
    
#     plt.figure(figsize=(9, 6))
#     sns.barplot(data=df_1st, x="GR_name", y=target_parameter, estimator=np.median, ci=None, dodge=False, linewidth=0.5, edgecolor="k", hue=target_color, hue_order=reversed(sorted(list(set(df_1st[target_color])))), palette=palette, order=list(sorted_nb.index))
#     plt.ylim(0, 2.5)
#     plt.xlim(-1, len(sorted_nb.head(100)))
#     plt.ylabel(target_parameter)
#     plt.tick_params(
#         axis='x',          # changes apply to the x-axis
#         which='both',      # both major and minor ticks are affected
#         bottom=False,      # ticks along the bottom edge are off
#         top=False,         # ticks along the top edge are off
#         labelbottom=False) # labels along the bottom edge are off
    
#     plt.tight_layout()
#     plt.savefig("{}/groups_ranked_{}_colorby_{}.pdf".format(figures_folder, target_parameter, target_color))

#     plt.show()
#     plt.close()
    
    # Plot overall view, without xlabels
    plt.figure(figsize=(15, 5))
    sns.boxplot(x='GR_name', y=target_parameter, fliersize=2, hue= target_color, hue_order=reversed(sorted(list(set(df_1st[target_color])))), palette = palette, data = df_1st, dodge=False, order=list(sorted_nb.index), linewidth=1)
    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=False,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=False) # labels along the bottom edge are off
    plt.xlabel('')
    
    if 'normalised' in target_parameter:
        plt.ylim(0, 100)
    else:
        plt.ylim(0, plt.gca().get_ylim()[1])
        
    plt.title('General boxplot of {}, with groups colored based on {}'.format(target_parameter, target_color))
    plt.tight_layout()
    plt.savefig("{}/groups_boxplot_{}_colorby_{}.pdf".format(figures_folder, target_parameter, target_color))

    plt.show()
    plt.close()
    
    # Now zoom in the top N
    top_n = sorted_nb.head(topn)
    df_top_n = df_1st.loc[df_1st.GR_name.isin(list(top_n.index))]
    color_values = reversed(sorted(list(set(df_1st[target_color]))))
    
    accepted_values = [i for i in color_values if i != '?']
    fig, ax = plt.subplots(1, len(accepted_values)+1, figsize=(5*(len(accepted_values)+1), 8), constrained_layout=True)
    
    sns.boxplot(ax=ax[0], fliersize=2, x='GR_name', y=target_parameter, hue= target_color, hue_order=reversed(sorted(list(set(df_1st[target_color])))), palette = palette, data = df_top_n, dodge=False, order=list(top_n.index), linewidth=1)
    ax[0].set_xticklabels(ax[0].get_xticklabels(), rotation='vertical')
    ax[0].set_title('Top {} groups in general'.format(topn))
    
    if 'normalised' in target_parameter:
        ax[0].set_ylim(0, 100)
    else:
        if ax[0].get_ylim()[1] > 4:
            ax[0].set_ylim(0, 4)
        else:
            ax[0].set_ylim(0, ax[0].get_ylim()[1])
    
    # And now into the topN by target category, excluding those with an unknown (?) category
    for i, value in enumerate(accepted_values):
        curr_df = df_1st.loc[df_1st[target_color] == value]
        sorted_nb = curr_df.groupby(['GR_name'])[target_parameter].median().sort_values(ascending=False)
        top_n = sorted_nb.head(topn)
        print(top_n)
        df_top_n = curr_df.loc[curr_df.GR_name.isin(list(top_n.index))]
        
        sns.boxplot(ax=ax[i+1], fliersize=2, x='GR_name', y=target_parameter, hue=target_color, hue_order=reversed(sorted(list(set(df_1st[target_color])))), palette = palette, data = df_top_n, dodge=False, order=list(top_n.index), linewidth=1)
        ax[i+1].set_xticklabels(ax[i+1].get_xticklabels(), rotation='vertical')       
        ax[i+1].set_title('Top {} groups for {}:\n{}'.format(topn, target_color, value))
        
        ax[i+1].set_ylim(ax[0].get_ylim()[0], ax[0].get_ylim()[1])
    
#     fig.title('General boxplot of {}, with groups colored based on {}'.format(target_parameter, target_color))
    
    plt.savefig("{}/groups_boxplot_{}_colorby_{}_top{}.pdf".format(figures_folder, target_parameter, target_color, topn))

    plt.show()
    

### 2.1. Rank groups by the overall quality of their 1st model

Done only for the groups that submitted models for at least 10 targets.

In [ ]:
target_parameters = ['S_geom_casp14', 'S_geom_casp13']
target_color = ['GR_type', 'GR_DeepL','GR_classification']

# target_parameters = ['S_geom_casp14']
# target_color = ['GR_type']

for p in target_parameters:
    for c in target_color:
        print('CURRENT PARAMETER: {}'.format(p))
        print('COLORED BY:        {}'.format(c))
        groups_boxplot(df_1st, p, c)

### 2.2. Check the effect of DipDiff on ranking 

Done only for the groups that submitted models for at least 10 targets.

In [ ]:
target_groups = ['ALPHAFOLD2', 'BAKER', 'BAKER-EXPERIMENTAL', 'BAKER-ROSETTASERVER', 'FEIG-R3', 'FEIG-R2', 'ZHANG', 'PROQ3D', 'PROQ2', 'VEROCNN-SELECT', 'P3DE']
color_map = 'bwr_r'
target_parameter = 'Model_DipDiff'

plt.figure(figsize=(5,4))
sorted_nb = df_1st.groupby(['GR_name'])[target_parameter, 'GDT_HA', 'S_geom_casp14', 'S_geom_casp13'].median()

sorted_nb = sorted_nb.sort_values(by='S_geom_casp14', ascending=False)
sorted_nb['CASP14_rank'] = range(1, len(sorted_nb)+1)

sorted_nb = sorted_nb.sort_values(by='S_geom_casp13', ascending=False)
sorted_nb['CASP13_rank'] = range(1, len(sorted_nb)+1)

sorted_nb['rank_diff'] = sorted_nb['CASP13_rank'] - sorted_nb['CASP14_rank']
#norm = plt.Normalize(-max(sorted_nb['rank_diff'].min(), sorted_nb['rank_diff'].max()), max(sorted_nb['rank_diff'].min(), sorted_nb['rank_diff'].max()))
norm = plt.Normalize(-min(sorted_nb['rank_diff'].min(), sorted_nb['rank_diff'].max()), min(sorted_nb['rank_diff'].min(), sorted_nb['rank_diff'].max()))

ax = plt.scatter(sorted_nb['S_geom_casp13'],sorted_nb[target_parameter], c=sorted_nb['rank_diff'], cmap=color_map, norm=norm, edgecolors='k', linewidth=0.5, zorder=2)
plt.hlines(y=0, xmin=0, xmax=100, linestyle=':', color='k', zorder=1)
plt.xlim(-0.1,2.5)
plt.ylim(min(df_1st[target_parameter]),max(df_1st[target_parameter]))
plt.xlabel('Median method S_geom_casp13')
plt.ylabel('Median method {}'.format(target_parameter))
# Remove the legend and add a colorbar
sm = plt.cm.ScalarMappable(cmap=color_map, norm=norm)
sm.set_array([])
ax.figure.colorbar(sm)
plt.tight_layout()
plt.savefig("{}/median_CASP13_vs_{}_colorby_rank_channge.pdf".format(figures_folder, target_parameter))
plt.show()
plt.close()

# MAKE THE ZOOM
plt.figure(figsize=(8,4))
ax = plt.scatter(sorted_nb['S_geom_casp13'],sorted_nb[target_parameter], c=sorted_nb['rank_diff'], cmap=color_map, norm=norm, edgecolors='k', linewidth=0.5, zorder=2)
plt.hlines(y=0, xmin=0, xmax=100, linestyle=':', color='k', zorder=1)
plt.xlim(0.6,1.0)

if target_parameter == 'Model_DipDiff':
    plt.ylim(-0.1, 0.05)
else:
    plt.ylim(0.2, 1.2)
    
plt.xlabel('Median method S_geom_casp13')
plt.ylabel('Median method {}'.format(target_parameter))
# Remove the legend and add a colorbar
sm = plt.cm.ScalarMappable(cmap=color_map, norm=norm)
sm.set_array([])
ax.figure.colorbar(sm)
plt.tight_layout()
plt.savefig("{}/median_CASP13_vs_{}_colorby_rank_channge_zoom.pdf".format(figures_folder, target_parameter))
plt.show()
plt.close()

sorted_nb.sort_values(by='S_geom_casp14', ascending=False).head(50)

## 3. Plot bar plots separated by target difficulty for the top 10 groups

Done only for the groups that submitted models for at least 10 targets.

In [ ]:
def groups_barplots(df_top1, parameter, rank_df = None, rank_parameter = 'S_geom_casp14', topn = 10, palette = 'Blues_r', logy = False):
    
    print('Ranked by: {}'.format(rank_parameter))
    
    if rank_df is None:
        rank_df = df_top1
        
    sorted_nb = rank_df.groupby(['GR_name'])[rank_parameter].median().sort_values(ascending=False).head(topn)
    
    df_top1['Target_classification'] = df_top1['Target_classification'].fillna('NaN')
        
    plt.figure(figsize=(15, 3))
    g = sns.barplot(data=df_top1.loc[df_top1.Target_classification != 'NaN'], x="Target_classification", y=parameter, errwidth=0.5, capsize=.02, estimator=np.median, order=['TBM-easy', 'TBM-hard', 'FM/TBM', 'FM'], ci='sd', linewidth=0.5, edgecolor="k", hue="GR_name", palette=palette, hue_order=list(sorted_nb.index))

    plt.ylabel('Median {}'.format(parameter))
    plt.xlabel('Target Classification')
    plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.)    
    
    if 'LLG' in parameter:
        cut = 60
        if 'normalised' in parameter:
            cut = cut*100/max(df_top1['deltaLLG'])
        else:
            g.axhline(cut, linestyle=':', color='k')
    else:
        g.axhline(0, linestyle=':', color='k')
    
    if 'GD' in parameter:
        plt.ylim(0, 100)
    elif 'DipDiff' in parameter:
        plt.ylim(min(plt.ylim()), max(plt.ylim()))
    else:
        plt.ylim(0, max(plt.ylim()))
    
    if logy:
        plt.yscale('log')
        plt.ylim(0.001, max(plt.ylim()))
        
    plt.tight_layout()
    
    if logy:
        plt.savefig("{}/groups_difficulty_barplots_{}_top{}_rankedby_{}_logscale.pdf".format(figures_folder, parameter, rank_parameter, topn))
    else:
        plt.savefig("{}/groups_difficulty_barplots_{}_top{}_rankedby_{}.pdf".format(figures_folder, parameter, topn, rank_parameter))
    plt.show()

In [ ]:
target_parameters = ['S_geom_casp14', 'GDT_TS', 'GDT_HA', 'GDC_SC', 'Model_DipDiff', 'Model_BBscore', 'Model_SCscore']
for p in target_parameters:
    print('CURRENT PARAMETER: {}'.format(p))
    groups_barplots(df_1st, p, palette = 'Spectral')

# 4. Check which parameters correlate with each other

In [ ]:
param_df = df.drop(['GR#', 'Target_title', 'Target', 'Model_GR_rank', 'Target_difficulty', 'Target_Neff', 'err', '#Residues', 'Model_CaBLAMdiff', 'LLG_const_B', 'Model_ChiDiff'], axis='columns')
z_columns = [i for i in param_df.columns if i.startswith('Z_') or 'casp' in i or 'Original' in i or 'from_' in i or 'PC' in i or 'Rms' in i or 'corr' in i or 'GR' in i]
for col in z_columns:
    param_df = param_df.drop([col], axis='columns')

param_df

In [ ]:
params_corr = param_df.corr(method='pearson')
print(params_corr['Model_DipDiff'].sort_values().tail())
print(params_corr['Model_DipDiff'].sort_values().head())

In [ ]:
casp14_param = ['LDDT', 'CAD_AA', 'SphGr', 'Model_SCscore', 'MolPrb_clash', 'Model_BBscore', 'Model_DipDiff', 'GDT_HA', 'QSE']
casp14_param = param_df[casp14_param]
plt.figure(figsize=(30, 30))
sns.clustermap(casp14_param.corr(), cmap = 'RdBu_r', center=0, vmin = -1, vmax=1, annot=True).savefig("{}/correlation_matrix_Scasp14_measures.pdf".format(figures_folder))

In [ ]:
plt.figure(figsize=(30, 30))
sns.clustermap(params_corr, cmap = 'RdBu_r', center=0, vmin = -1, vmax=1).savefig("{}/correlation_matrix_all_measures.pdf".format(figures_folder))

# 5. Compare CASP14 and CASP13 scoring function

In [ ]:
def get_colors(values, cmap = 'Blues_r'):
    
    colors = {}
    
    cmap = matplotlib.cm.get_cmap(cmap)
    norm = matplotlib.colors.Normalize(vmin=0, vmax=len(values))
    
    colours = [cmap(norm(i)) for i in range(len(values))]
    
    for i, value in enumerate(values):
        colors[value] = colours[i]
    
    return colors
    
def compare_casp_functions(df_1st, parameter_x, parameter_y, colorby='GR_classification', groupby = 'GR#', equal_axis = True):
    
    medians = df_1st.groupby([groupby]).median()
    colors = []
    for index, row in medians.iterrows():
        tmp = df_1st[df_1st[groupby] == index][colorby]
        colors.append(tmp[tmp.index[0]])
    medians[colorby] = colors
    
#     colors = get_colors(sorted(set(list(df_1st[colorby]))), cmap = 'Blues')
#     gr_colors = {gr: colors[list(set(df_1st[df_1st[groupby] == gr][colorby]))[0]] for gr, row in medians.iterrows()}
    
    plt.clf()
    plt.figure(figsize=(7, 7))
    sns.scatterplot(parameter_x, parameter_y, data = medians, hue = colorby, s=30, edgecolor='k', hue_order=sorted(list(set(colors)))[::-1], palette = 'Blues_r')
    
#    for index, row in medians.iterrows():
        #plt.scatter(row[parameter_x], row[parameter_y],  c=np.array([gr_colors[index]]), edgecolors='k', s=30)
        #plt.text(row[parameter_x], row[parameter_y], index, c=gr_colors[index], ha='center', va='center')
    
    if equal_axis:
        
        plt.plot([0, 2.5], [0, 2.53], ls=":", c='grey')
        plt.xlim(0, 2.5)
        plt.ylim(0, 2.5)
    
    plt.ylabel(parameter_y)
    plt.xlabel(parameter_x)
    
    plt.savefig('{}/scatter_groups_ranking_{}_vs_{}_colorby_{}.pdf'.format(figures_folder,parameter_x, parameter_y, colorby))
    
    plt.show()

In [ ]:
target_columns = ['S_geom_casp13', 'S_geom_casp14']

for i, x in enumerate(target_columns):
    for j, y in enumerate(target_columns):
        if i < j:
            compare_casp_functions(df_1st, x, y, groupby='GR_name', colorby='GR_type')
    

# 6. Check accuracy accross CASPs

In [ ]:
def compare_casps(parameter, cancelled_targets = cancelled_targets, which_casps = [12,13,14], exclude_group = None, difficulty = 'Target_difficulty'):

    tmp = {difficulty:[], parameter: [], 'CASP': [], 'Target_type': []}
    
    for casp in which_casps:
        if casp != 14:
            cancelled_targets = []
        else:
            cancelled_targets = cancelled_targets
            
        df = load_casp_zscore_table(casp, cancelled_targets, domains_only = True)
        df = df[df.Model.str.startswith('T')]
        
        for target in set(list(df.Target)):
            tmp_df = df.loc[(df.Target == target)].sort_values(by='GDT_HA', ascending=False)
            
            if casp == 14:
                tmp_df = tmp_df.loc[(df['GR#'] != exclude_group)]
                
            target_type = list(tmp_df.Target_classification)[0]
            try:
                target_type = target_type.split('-')[0]
            except:
                pass
            
            tmp[parameter].append(list(tmp_df[parameter])[0])
            tmp[difficulty].append(list(tmp_df[difficulty])[0])
            tmp['CASP'].append(str(casp))
            tmp['Target_type'].append(target_type)

    tmp = pd.DataFrame(tmp)
    
    if difficulty == 'Target_difficulty':
        tmp[difficulty] = 100-tmp[difficulty]
    
    print('Exclude Group {}'.format(exclude_group))
    
    plt.figure(figsize=(15, 3))
    sns.lmplot(x=difficulty, y=parameter, data=tmp, hue='CASP', markers=[".", "x", '+'], lowess=True, palette='nipy_spectral_r', height=3, aspect=1) 
    if difficulty == 'Target_difficulty':
        plt.xlim(0,100)
    else:
        plt.xscale('log')
    
    if 'GD' in parameter:
        plt.ylim(0,100)
    
    plt.tight_layout()
    
    if exclude_group is None:
        plt.savefig("{}/CASPs_{}_vs_{}_comparison.pdf".format(figures_folder, difficulty, parameter))
    else:
        plt.savefig("{}/CASPs_{}_vs_{}_comparison-excluding_{}.pdf".format(figures_folder, difficulty, parameter, exclude_group))
    plt.show()
    plt.close()

In [ ]:
paremeters = ['GDT_HA', 'Model_BBscore', 'Model_SCscore', 'Model_DipDiff']
for parameter in paremeters:
    compare_casps(parameter)
    compare_casps(parameter, exclude_group='427')
    
# for parameter in paremeters:
#     compare_casps(parameter, difficulty = 'Target_Neff')
#     compare_casps(parameter, difficulty = 'Target_Neff', exclude_group='427')